## Import packages


In [1]:
import pandas as pd
import nltk
import numpy
import spacy
import textacy
import re
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from gensim.models import ldamulticore
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models, similarities, matutils
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import RegexpTokenizer
from nltk.stem import LancasterStemmer, PorterStemmer

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.utils import simple_preprocess


import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### importing data 

In [2]:
df_chats=pd.read_csv('C:/Users/52100722/Desktop/nidhi/chatlog_honeywell/CHATTRANSCRIPT_20181205_1200.csv',
                    delimiter='\t', encoding='1252')
df_chats.dropna(inplace=True)

### Data preperation

In [7]:
## Dropping messages where origin == system
df_no_system=df_chats[df_chats.Origin!='SYSTEM']
df_no_system.head()

,UniqueID,SessionID,ServiceLine,Date/Time,Origin,Message
3,4,144209,Honeywell US,03:45.0,H Eugene Talbott ...,H&I
7,8,144209,Honeywell US,03:55.0,Virgilio Dela Pena,Thank you for chatting with us today. How can ...
8,9,144209,Honeywell US,05:05.0,H Eugene Talbott ...,Good Morning Virgilio - I have a case open on ...
11,12,144209,Honeywell US,05:17.0,H Eugene Talbott ...,And am checking in on the case to see what the...
12,13,144209,Honeywell US,05:18.0,Virgilio Dela Pena,I'll be happy to assist you with that. It will...


In [9]:
df_honeywell=df_no_system[df_no_system.ServiceLine=="Honeywell US"]
df_honeywell.drop(columns=["UniqueID","Date/Time","ServiceLine","Origin"],inplace=True)
df_honeywell["Message"]=df_honeywell["Message"].astype(str)
df_honeywell=df_honeywell.groupby(["SessionID"]).sum()["Message"]

C:\Users\52100722\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [11]:
doc_honeywell=[]
for i in range(len(df_honeywell)):
    doc_honeywell.append(df_honeywell.iloc[i])
    
doc_honeywell=[i.lower() for i in doc_honeywell]

In [13]:
df_agent=df_chats[df_chats['Message'].str.contains(' How can I help you?')]
name_agents=(pd.unique(df_agent["Origin"]))
df_qna=pd.concat([df_no_system, df_agent, df_agent]).drop_duplicates(keep=False)

### tokenizing the document 

In [14]:
lemmer=WordNetLemmatizer()
stemmer = PorterStemmer()
regex=RegexpTokenizer("\\b[a-zA-Z][a-zA-Z][a-zA-Z]+\\b")

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        #stemmed.append(stemmer.stem(item))
        stemmed.append(lemmer.lemmatize(item,"v"))
    return stemmed


def tokenize(text):
    #text=text.lower()
    text=re.sub("[\(\[].*?[\)\]]", "", text)
    tokens = regex.tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

stop  = ['thank', 'how', 'good', 'morning','this', 'can', 'okay','cayou',
          'miss', 'spallo', 'would', 'so', 'yes', 'the','join', 'session', 
           'assign', 'new', 'create', 'accept', 'chat','h&ithank you for chatting with us today.','how can i help you?'
            'access', 'new','click','site','since','thanks','happy','david',"acrobat reader","actionno porblemok",
             'already','like','send','yes','last','right','also','Virgilio',''
             ,'give','know','would','get','make','wonderful','phone','heard','aboutthe',
              'apologize','enjoy','welcome','assist','actually','active'
             'want','something','heard','see','would','may','else','waiting','web','scroll'
             'one','day','minute','see word','today','inform','using','during','beconnected','cal return'
             'call','good','morninig','anything','need','chat','thank','calling','please','moment',
             'check','welcome','nan','assigned','session','can','help','still','session','assist',
             'number','checking','chatting','time','change','acknowledgment',"customer","hello",
              "just","minute","send","use","try","able","clickyou","above","absolutely","share","connect",]

from sklearn.feature_extraction import text 
stop_wd = text.ENGLISH_STOP_WORDS.union(stop,name_agents)
stp_wd=([i.lower() for i in stop_wd])

In [17]:
doc_token=[tokenize(doc_honeywell[i]) for i in range(len(doc_honeywell))]
dictionary=gensim.corpora.Dictionary(doc_token)

### Removing stop words by vectorising the stop tokens

In [20]:
stop_id=[dictionary.token2id[i] for i in stp_wd if i in dictionary.token2id ]       
dictionary.filter_tokens(stop_id)
dictionary.compactify()
corpus=[dictionary.doc2bow(i) for i in doc_token]


In [24]:
tfidf=models.TfidfModel(corpus)
corpus_tfidf=tfidf[corpus]

2019-05-29 10:08:29,401 : INFO : collecting document frequencies
2019-05-29 10:08:29,405 : INFO : PROGRESS: processing document #0
2019-05-29 10:08:29,496 : INFO : PROGRESS: processing document #10000
2019-05-29 10:08:29,535 : INFO : calculating IDF weights for 13913 documents and 32061 features (528820 matrix non-zeros)


###  LSI model 

In [27]:
lsi=models.LsiModel(corpus=corpus_tfidf,num_topics=15,id2word=dictionary)

2019-05-29 10:09:06,474 : INFO : using serial LSI version on this node
2019-05-29 10:09:06,475 : INFO : updating model with new documents
2019-05-29 10:09:10,981 : INFO : preparing a new chunk of documents
2019-05-29 10:09:11,237 : INFO : using 100 extra samples and 2 power iterations
2019-05-29 10:09:11,238 : INFO : 1st phase: constructing (32062, 115) action matrix
2019-05-29 10:09:11,422 : INFO : orthonormalizing (32062, 115) action matrix
2019-05-29 10:09:12,471 : INFO : 2nd phase: running dense svd on (115, 13913) matrix
2019-05-29 10:09:12,742 : INFO : computing the final decomposition
2019-05-29 10:09:12,745 : INFO : keeping 15 factors (discarding 51.548% of energy spectrum)
2019-05-29 10:09:12,768 : INFO : processed documents up to #13913
2019-05-29 10:09:12,771 : INFO : topic #0(21.840): -0.190*"surcharge" + -0.154*"benefit" + -0.150*"contact" + -0.146*"center" + -0.146*"screen" + -0.140*"biometric" + -0.136*"coverage" + -0.133*"ithank" + -0.129*"hear" + -0.127*"event"
2019-05

In [29]:
lsi_topic=lsi.show_topics()
df_topic_terms=pd.DataFrame(lsi_topic,columns=["topic_number","terms"])
df_topic_terms.head()

,topic_number,terms
0,0,"-0.190*""surcharge"" + -0.154*""benefit"" + -0.150..."
1,1,"0.985*""ithank"" + 0.073*""hear"" + -0.045*""surcha..."
2,2,"-0.397*""surcharge"" + -0.287*""biometric"" + -0.2..."
3,3,"0.803*""hear"" + 0.386*""assume"" + 0.334*""assista..."
4,4,"-0.456*""card"" + -0.357*""dental"" + -0.209*""metl..."


## Recommender System

### Identify  similar documents related to user's inputs : LSI model

In [100]:
doc=df_honeywell.values[0:-1]

In [101]:
## Transform our corpus to lsi space and index it ##
index=similarities.MatrixSimilarity(lsi[corpus])

## now converting user's document into lsi space vector ##
usr_doc= input()
vec_bow=dictionary.doc2bow(usr_doc.lower().split())
usr_lsi_vec=lsi[vec_bow]

## now find similarity of user vector aginst our indexed document ##
sims=index[usr_lsi_vec]

## sort the similarity ##
siml=sorted(enumerate(sims), key=lambda val: -val[1])[:6]
for sim_doc_id,sim_score in siml:
    print("\033[1m"+"\nScore: " + str(sim_score))
    print("Document: " + doc[sim_doc_id] )

2019-05-28 11:54:10,894 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2019-05-28 11:54:11,168 : INFO : creating matrix with 13913 documents and 15 features


account info

Score: 0.97745323
Document: H&IHi I enrolled in my benefits but it said that I need a bank account with bank of america to get my HSAThank you for chatting with us today. How can I help you?Can you help me set up account with bank of america for HSA?I'll be happy to assist you with that. It will be just a minute while I access your records.I call bank of america and they said that this is done through honeywellLet just go ahead and check your account first.Just give me a moment please.Thank you for waiting Ms. Patterson.Since you have elected already your HSA, your Eligibility Information will be forwarded to Bank of America and a Welcome Kit will be sent to you by mail to open the account.what is in the welcome kit and why do I have to wait for the welcome kit to open the accountyou there?Welcome kit is your guide to your Health Saving Account.You need to wait for that because Eligibility Information is being process first until Bank of America received it.ok thanksbye

